# Goal

1. Memory function : done
2. Load PDF function : done
3. Chain question function : 50%
4. Store the result into mysql function

In [1]:
import os
from langchain.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from utils import read_json
from langchain_core.output_parsers import StrOutputParser
from langchain import LLMChain
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
import json

In [3]:
configs = read_json('configs.json')
DATA_PATH = configs["DATA_PATH"]
DB_PATH = configs['DB_PATH']
files_list = [file for file in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, file))]
files_list
def read_pdf(data_path=DATA_PATH, files_list=files_list):
    
    for file in files_list:
        loader = PyPDFLoader(data_path + file)
        content = loader.load()
    return content
pdf_content = read_pdf()

In [12]:
len(pdf_content)

15

In [4]:
llm = ChatOllama(
    model = 'llama3',
    verbose = 'true',
)

In [9]:
len(pdf_content[0].page_content)

635

In [22]:
summary = ""
i = 0

while i < 3:
    print(i, len(pdf_content[i].page_content))
    messages = [
        SystemMessage(
            content="你是一個專業的中文AI助手，用繁體中文回答所有的問題，即使問題是英文也要用中文回答"
            ), 
        HumanMessage(
            content=f"對以下文件作內容摘要整理:{pdf_content[0].page_content}"
        )
    ]
    model_response = llm.invoke(messages)

    summary += '\n' + model_response.content

    i+=1

0 635
1 869
2 852


In [23]:
summary.

'\n**Content Summary:**\n\n**Case:** Rectification Case for Land Development Issues around Taiwan Taoyuan International Airport A7 Station\n\n**Main Parties Involved:** Ministry of the Interior and Taoyuan City Government\n\n**Background:** The case involves issues with land development around Taiwan Taoyuan International Airport A7 Station, which was undertaken by the Ministry of the Interior using outdated maps. This led to uncertainties in the location of measurement points, making it impossible to proceed with related engineering projects.\n\n**Key Points:**\n\n1. The Ministry of the Interior used outdated maps to handle urban planning changes and zone adjustments for land development around A7 Station.\n2. This resulted in inconsistencies between urban planning boundaries and property boundaries, leading to measurement point uncertainties, delayed project completion, and increased costs.\n3. The total cost of these issues reached NT$98 million.\n\n**Legal Basis:**\n\n1. Urban Plan

In [19]:
summary = ""

for i, pages in enumerate(pdf_content):
    print(i, len(pages.page_content))
    messages = [
        SystemMessage(
            content="你是一個專業的中文AI助手，用繁體中文回答所有的問題，即使問題是英文也要用中文回答"
            ), 
        HumanMessage(
            content=f"對以下文件作內容摘要整理:{pdf_content[0].page_content}"
        )
    ]
    model_response = llm.invoke(messages)

    summary += '\n' + model_response.content

0 635
1 869
2 852
3 672
4 668
5 783
6 822
7 962
8 944
9 922
10 813
11 727
12 706
13 645
14 266


In [8]:
messages = [
    SystemMessage(
        content="你是一個專業的中文AI助手，用繁體中文回答所有的問題，即使問題是英文也要用中文回答"
        ), 
    HumanMessage(
        content=f"對以下文件作內容摘要整理:{pdf_content[0].page_content}"
    )
]
model_response_test = llm.invoke(messages)

In [10]:
model_response_test

AIMessage(content='**糾正案文**\n\n**被糾正機關：** 內政部、桃園市政府\n\n**案由：** 內政部使用老舊、過時圖資辦理臺灣桃園國際機場捷運 A7站區周邊土地開發，導致測釘樁位產生疑義、相關工程無法施作。同時，桃園市政府任令本案土地長期被棄置廢棄物，累計高達 52萬餘立方公尺。\n\n**事實與理由：**\n\n1. 內政部使用老舊、過時圖資辦理本案都市計畫變更和區段徵收作業，造成都市計畫範圍與徵收地籍界線不一致，導致測釘樁位產生疑義、相關工程無法施作。\n2. 本案土地長期被棄置廢棄物，累計高達 52萬餘立方公尺，造成施工後出現諸多問題，大幅延宕完工期程，並增加開發成本新臺幣 9,801萬餘元。\n\n**理由：**\n\n1. 內政部違反「都市計畫法」第27條第1項第4款規定，未進行實際情況的變更。\n2. 內政部違反「土地徵收條例」第4條第1項第3款規定，未將都市土地之農業區、保護區變更為建築用地或工業區變更為住宅區、商業區。\n3. 內政部召開研商會議，並於民國99年4月13日函發作，但仍造成糾正案的需要。', response_metadata={'model': 'llama3', 'created_at': '2024-05-09T07:05:21.7809798Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 264089400400, 'load_duration': 8625353700, 'prompt_eval_count': 640, 'prompt_eval_duration': 134236556000, 'eval_count': 399, 'eval_duration': 121221998000}, id='run-79a8ee73-1ff5-46d0-8b63-b7de9dd89fad-0')

In [11]:
model_response_test.content

'**糾正案文**\n\n**被糾正機關：** 內政部、桃園市政府\n\n**案由：** 內政部使用老舊、過時圖資辦理臺灣桃園國際機場捷運 A7站區周邊土地開發，導致測釘樁位產生疑義、相關工程無法施作。同時，桃園市政府任令本案土地長期被棄置廢棄物，累計高達 52萬餘立方公尺。\n\n**事實與理由：**\n\n1. 內政部使用老舊、過時圖資辦理本案都市計畫變更和區段徵收作業，造成都市計畫範圍與徵收地籍界線不一致，導致測釘樁位產生疑義、相關工程無法施作。\n2. 本案土地長期被棄置廢棄物，累計高達 52萬餘立方公尺，造成施工後出現諸多問題，大幅延宕完工期程，並增加開發成本新臺幣 9,801萬餘元。\n\n**理由：**\n\n1. 內政部違反「都市計畫法」第27條第1項第4款規定，未進行實際情況的變更。\n2. 內政部違反「土地徵收條例」第4條第1項第3款規定，未將都市土地之農業區、保護區變更為建築用地或工業區變更為住宅區、商業區。\n3. 內政部召開研商會議，並於民國99年4月13日函發作，但仍造成糾正案的需要。'

In [16]:
messages = [
    SystemMessage(
        content="你是一個專業的中文AI助手，用繁體中文回答所有的問題，不知道就說不知道，不做過多的臆測"
        ),
    HumanMessage(
        content=f"閱讀以下整份資料，給我摘要描述:{pdf_content}，用繁體中文回覆。"
    )
]
hat_model_response = llm.invoke(messages)

In [17]:
hat_model_response

AIMessage(content="Based on the provided documents, I will summarize the issues and provide a response in Traditional Chinese:\n\n**Summary**\n\nThe Ministry of the Interior (MOI) had planned to develop a land area in Taoyuan City, but due to various issues, including the discovery of massive amounts of waste, the project was delayed by 137 days. The MOI also found that the original plan for residential use was not suitable, and the area would need to be zoned as an agricultural or protected zone instead.\n\n**Response**\n\nThe MOI's handling of this project is a classic example of inefficiency and poor planning. By using outdated maps and failing to conduct thorough investigations beforehand, the MOI has caused significant delays and increased costs for the project.\n\nFirstly, the discovery of massive amounts of waste during construction was completely unexpected. This highlights the importance of conducting proper site surveys and environmental assessments before embarking on large-

In [15]:
init_prompt = ChatPromptTemplate.from_template('你是一個熟悉司法判決內容的專家，用繁體中文回答以下問題')
prompt_1 = ChatPromptTemplate.from_template('先仔細閱讀以下這份文件{content}。告訴我總共可以分成那些段落? ')
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key='phase1_ans')

prompt_2 = ChatPromptTemplate.from_template('這份文件可以分成以下這些段落:{phase1_ans}。給我每一個段落的摘要')
chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key='summaries')

overall_chain = SequentialChain(
    chains=[chain_1, chain_2]
    input_variables=['content']
    output_variables=['phase1_ans', 'summaries']
    verbose=True
)

result = overall_chain({'content' : content})